In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import datetime as dt
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
# Training data sets
benef = pd.read_csv('./data/Train_Beneficiary.csv')
inpat = pd.read_csv('./data/Train_Inpatient.csv')
outpat = pd.read_csv('./data/Train_Outpatient.csv')
fraud = pd.read_csv('./data/Train-Potential Fraud.csv')

In [106]:
print(benef.shape)
print(inpat.shape)
print(outpat.shape)
print(fraud.shape)

(138556, 25)
(40474, 31)
(517737, 28)
(5410, 2)


In [3]:
#Create columns for inpatient and outpatient data.
inpat['patientType'] = np.repeat('inpatient', len(inpat))
outpat['patientType'] = np.repeat('outpatient', len(outpat))

In [4]:
#Combining the inpatient and outpatient data
patient = pd.concat([inpat, outpat], axis=0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [5]:
# Combining patient dataset with beneficiary dataset 
full_data =pd.merge(patient, benef, on='BeneID', how='left')

In [6]:
full_data1 = full_data.copy()

In [7]:
# Creating DaysAdmitted Feature by substracting claim start date from claim end date 
full_data1['ClaimStartDt'] = pd.to_datetime(full_data1['ClaimStartDt'])
full_data1['ClaimEndDt'] = pd.to_datetime(full_data1['ClaimEndDt'])
full_data1['DaysAdmitted'] = full_data1['ClaimEndDt'] - full_data1['ClaimStartDt']

In [8]:
# Chaging DaysAdmitted into integer object 
full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].astype(str)
full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].str.replace('days', ' ')

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted']. \
str.replace('00:00:00.000000000', '')

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].astype(int)

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'] + 1

#### First Feature Engineered: DaysAdmittedMean

In [9]:
# Creating a separate dataframe of DaysAdmittedMean by Provider
days = pd.DataFrame(full_data1.groupby('Provider')['DaysAdmitted'].mean())
days = days.reset_index()

In [10]:
# Merging Potential Fraud dataset with DaysAdmittedMean dataset 
fraud1 = fraud.copy()
fraud2 = pd.merge(fraud1, days, on='Provider')

#### 2nd Feature: DeductibleAmountPaid Mean

In [11]:
DeductibleMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['DeductibleAmtPaid'].mean())

fraud2 = pd.merge(fraud2, DeductibleMean, on='Provider')

In [12]:
fraud2.groupby('PotentialFraud')['DeductibleAmtPaid'].mean()

PotentialFraud
No     135.498511
Yes    350.839395
Name: DeductibleAmtPaid, dtype: float64

#### 3rd Feature: Insurance Claim Amount Reimbursed Mean

In [13]:
InscReimbursed = pd.DataFrame(full_data1.groupby('Provider') \
                              ['InscClaimAmtReimbursed'].mean())

fraud2 = pd.merge(fraud2, InscReimbursed, on='Provider')

In [14]:
fraud2.groupby('PotentialFraud')['InscClaimAmtReimbursed'].mean()

PotentialFraud
No     1523.780804
Yes    3842.795104
Name: InscClaimAmtReimbursed, dtype: float64

#### 4th Feature: Insurance Covered Percentage Mean

In [15]:
full_data1['InscCoveredPercent'] = full_data1['InscClaimAmtReimbursed'] \
/(full_data1['InscClaimAmtReimbursed'] + full_data1['DeductibleAmtPaid'])

InscCoveredPercent = pd.DataFrame(full_data1.groupby('Provider') \
                                  ['InscCoveredPercent'].mean())

fraud2 = pd.merge(fraud2, InscCoveredPercent, on='Provider')

In [16]:
fraud2.groupby('PotentialFraud')['InscCoveredPercent'].mean()

PotentialFraud
No     0.962077
Yes    0.934837
Name: InscCoveredPercent, dtype: float64

#### 5th Feature: Total Claim Amount Mean

In [17]:
full_data1['TotalClaimAmount'] = full_data1['InscClaimAmtReimbursed']\
+ full_data1['DeductibleAmtPaid']

TotalClaimAmount = pd.DataFrame(full_data1.groupby('Provider') \
                                ['TotalClaimAmount'].mean())

fraud2 = pd.merge(fraud2, TotalClaimAmount, on='Provider')

In [18]:
fraud2.groupby('PotentialFraud')['TotalClaimAmount'].mean()

PotentialFraud
No     1645.778736
Yes    4142.438122
Name: TotalClaimAmount, dtype: float64

#### 6th Feature: Daily Total Charge

In [19]:
full_data1['DailyTotalCharge'] = full_data1['TotalClaimAmount'] \
/full_data1['DaysAdmitted']

DailyTotalCharge = pd.DataFrame(full_data1.groupby('Provider') \
                                ['DailyTotalCharge'].mean())

fraud2 = pd.merge(fraud2, DailyTotalCharge, on='Provider')

In [20]:
fraud2.groupby('PotentialFraud')['DailyTotalCharge'].mean()

PotentialFraud
No     471.273799
Yes    952.785225
Name: DailyTotalCharge, dtype: float64

#### Features 7-10: Annual Reimbursement and Deductible Paid Amounts for Inpatient and Outpatient

In [21]:
IPAnnReimbMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['IPAnnualReimbursementAmt'].mean())

IPAnnDeductMean = pd.DataFrame(full_data1.groupby('Provider') \
                               ['IPAnnualDeductibleAmt'].mean())

OPAnnReimbMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['OPAnnualReimbursementAmt'].mean())

OPAnnDeductMean = pd.DataFrame(full_data1.groupby('Provider') \
                               ['OPAnnualDeductibleAmt'].mean())

In [22]:
fraud2 = pd.merge(fraud2, IPAnnReimbMean, on='Provider')
fraud2 = pd.merge(fraud2, IPAnnDeductMean, on='Provider')
fraud2 = pd.merge(fraud2, OPAnnReimbMean, on='Provider')
fraud2 = pd.merge(fraud2, OPAnnDeductMean, on='Provider')

In [23]:
fraud2.groupby('PotentialFraud')['IPAnnualReimbursementAmt'].mean()

PotentialFraud
No     5868.975601
Yes    9052.076164
Name: IPAnnualReimbursementAmt, dtype: float64

In [24]:
fraud2.groupby('PotentialFraud')['IPAnnualDeductibleAmt'].mean()

PotentialFraud
No     638.107001
Yes    946.817676
Name: IPAnnualDeductibleAmt, dtype: float64

In [25]:
fraud2.groupby('PotentialFraud')['OPAnnualReimbursementAmt'].mean()

PotentialFraud
No     2191.723203
Yes    2273.113210
Name: OPAnnualReimbursementAmt, dtype: float64

In [26]:
fraud2.groupby('PotentialFraud')['OPAnnualDeductibleAmt'].mean()

PotentialFraud
No     628.840769
Yes    631.565539
Name: OPAnnualDeductibleAmt, dtype: float64

#### Feature 11: Total Annual Claim Amount Mean

In [27]:
full_data1['TotalAnnClaimAmt'] = full_data1['IPAnnualReimbursementAmt']+ \
full_data1['IPAnnualDeductibleAmt']+full_data1['OPAnnualReimbursementAmt'] \
+ full_data1['OPAnnualDeductibleAmt']

TotalAnnClaimAmt = pd.DataFrame(full_data1.groupby('Provider') \
                                ['TotalAnnClaimAmt'].mean())

fraud2 = pd.merge(fraud2, TotalAnnClaimAmt, on='Provider')

In [28]:
fraud2.groupby('PotentialFraud')['TotalAnnClaimAmt'].mean()

PotentialFraud
No      9327.646575
Yes    12903.572589
Name: TotalAnnClaimAmt, dtype: float64

## Randy's Features

#### Add number of doctors

In [30]:
doctors= full_data1.groupby(['Provider', 'AttendingPhysician']) \
['AttendingPhysician'].count().reset_index(name='NumOfDoctors'). \
groupby('Provider')['NumOfDoctors'].count().reset_index()

fraud2 = pd.merge(fraud2, doctors, on='Provider')

#### Add number of patients

In [31]:
patient= full_data.groupby(['Provider','BeneID'])['BeneID'].count(). \
reset_index(name='NumOfPatients').groupby('Provider')['NumOfPatients']. \
count().reset_index()

fraud2 = pd.merge(fraud2, patient, on='Provider')

#### Add Type of Service

In [32]:
a = full_data1.groupby(['Provider', 'patientType'])['BeneID'].count().reset_index(name='count').\
drop('count', axis=1)

a_list= list(zip(a.Provider, a.patientType))

providerDict= {}

for ele in a_list:
    if ele[0] not in providerDict:
        providerDict[ele[0]]= ele[1]    
    else:
        providerDict[ele[0]] = 'both'
        

providerService = pd.DataFrame(providerDict.keys(), providerDict.values()).reset_index().\
rename(columns={'index':'ServiceType', 0:'Provider'})

In [33]:
fraud2 = pd.merge(fraud2, providerService, on='Provider')

#### Add number of claims

In [34]:
claims= full_data1.groupby(['Provider', 'ClaimID'])['ClaimID'].count().\
reset_index(name='NumOfClms').groupby('Provider')['NumOfClms'].count().reset_index()

fraud2 = pd.merge(fraud2, claims, on='Provider')

In [64]:
fraud5 = fraud2.copy()

#### Add weekly claims 

In [37]:
full_data1['startClaimWeek']= full_data1['ClaimStartDt'].dt.week

In [38]:
weeklyClaims= full_data1.groupby(['Provider','startClaimWeek', 'ClaimID'])['ClaimID'].count().reset_index(name='WeeklyClaims').\
groupby(['Provider'])['WeeklyClaims'].count().reset_index()

In [65]:
fraud5= pd.merge(fraud5, weeklyClaims, on='Provider', how='left')

#### Add average duration of treatment per claims

#### Add monthly claims

In [42]:
full_data1['startClaimMonth'] = full_data1['ClaimStartDt'].dt.month

monthly = full_data1.groupby(['Provider','startClaimMonth', 'ClaimID'])['ClaimID'].count().reset_index(name='MonthlyClaims').\
groupby(['Provider'])['MonthlyClaims'].count().reset_index()

In [67]:
fraud5 = pd.merge(fraud5, monthly, on='Provider', how='left')

#### Add number of states each provider operates in

In [47]:
numStates = full_data1.groupby(['Provider', 'State'])['State'].count().reset_index(name='a').groupby('Provider')['State'].count().\
reset_index(name='numStates')

In [69]:
fraud5 = pd.merge(fraud5, numStates, on='Provider')

#### Add number of counties each provider operates in 

In [50]:
numCounties = full_data.groupby(['Provider', 'County'])['County'].count().reset_index(name='a'). \
groupby('Provider')['County'].count().reset_index(name='numCounties')

In [71]:
fraud5 = pd.merge(fraud5, numCounties, on='Provider')

In [73]:
fraud6 = fraud5.copy()

#### Add average number of chronic conditions per provider

In [55]:
patientChronic = full_data1.filter(regex='Chronic').replace(to_replace=2, value=0).sum(axis=1).\
reset_index(name='NumChronicCond')

In [56]:
chronic = pd.concat([patientChronic, full_data1[['Provider', 'ClaimID']]], axis=1)

In [58]:
avgChronic = chronic.groupby(['Provider', 'NumChronicCond'])['ClaimID'].count().reset_index(name='NumClaims').\
groupby('Provider').agg({'NumChronicCond':'mean', 'NumClaims':'mean'}).reset_index().\
rename(columns={'NumChronicCond':'AvgChronic', 'NumClaims':'AvgClaim'})

In [74]:
fraud6 = pd.merge(fraud6, avgChronic, on='Provider')

#### Add number of claim admit diagnosis codes per provider 

In [61]:
numDiffDiagnosisCode= full_data1.groupby(['Provider', 'ClmAdmitDiagnosisCode'])['ClmAdmitDiagnosisCode'].count(). \
reset_index(name='a').groupby('Provider')['ClmAdmitDiagnosisCode'].count().reset_index(name='numDiffDiagnosisCode')

In [76]:
fraud6 = pd.merge(fraud6, numDiffDiagnosisCode, on='Provider', how='left')

#### Add number of group diagnosis codes per provider

In [78]:
numDiffGroupDiagCode = full_data1.groupby(['Provider', 'DiagnosisGroupCode'])['DiagnosisGroupCode']. \
count().reset_index(name='a').groupby('Provider')['DiagnosisGroupCode']. \
count().reset_index(name='numDiffGroupDiagCode')

In [79]:
fraud6 = pd.merge(fraud6, numDiffGroupDiagCode, on='Provider', how='left')

## Jay's Features

In [81]:
fraud7 = fraud6.copy()

#### Adding average of patients' age per provider

In [83]:
# Calculating the age of the patient at the time of their service
full_data1['ClaimStartDt2'] =  pd.to_datetime(full_data1['ClaimStartDt'], format='%Y/%m/%d')
full_data1['DOB'] =  pd.to_datetime(full_data1['DOB'], format='%Y/%m/%d')
full_data1['AgeWhenServed'] = full_data1['ClaimStartDt'] - full_data1['DOB']

In [ ]:
# Converting the unit of AgeWhenServed to year from days
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'] / np.timedelta64(1, 'Y')

In [85]:
# Converting AgeWhenServed to int from float
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].fillna(-1)
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].astype(int)
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].replace(-1, np.nan)

In [86]:
avgage = full_data1.groupby(['Provider', 'BeneID'])['AgeWhenServed']. \
mean().reset_index(name = "AvgAgeWhenServed").dropna() \
.groupby('Provider')['AvgAgeWhenServed'].mean().reset_index()

In [87]:
fraud7 = pd.merge(fraud7, avgage, on='Provider', how='left')

#### Adding the count of Duplicated BeneID per Provider

In [89]:
inpatient = full_data1.loc[full_data1.patientType == 'inpatient']
outpatient = full_data1.loc[full_data1.patientType == 'outpatient']

In [ ]:
# Dataframe of duplicated data for inpatients
inpatient_d = inpatient[inpatient.duplicated(['ClmAdmitDiagnosisCode', \
                                              'DiagnosisGroupCode', \
                                              'ClmDiagnosisCode_1', \
                                              'ClmDiagnosisCode_2', \
                                              'ClmDiagnosisCode_3', \
                                              'ClmDiagnosisCode_4', \
                                              'ClmDiagnosisCode_5', \
                                              'ClmDiagnosisCode_6', \
                                              'ClmDiagnosisCode_7', \
                                              'ClmDiagnosisCode_8', \
                                              'ClmDiagnosisCode_9', \
                                              'ClmDiagnosisCode_10', \
                                              'ClmProcedureCode_1', \
                                              'ClmProcedureCode_2', \
                                              'ClmProcedureCode_3', \
                                              'ClmProcedureCode_4', \
                                              'ClmProcedureCode_5', \
                                              'ClmProcedureCode_6'], \
                                             keep = False)]

In [91]:
# Dataframe of duplicated data for inpatients
outpatient_d = outpatient[outpatient.duplicated(['ClmAdmitDiagnosisCode', \
                                              'DiagnosisGroupCode', \
                                              'ClmDiagnosisCode_1', \
                                              'ClmDiagnosisCode_2', \
                                              'ClmDiagnosisCode_3', \
                                              'ClmDiagnosisCode_4', \
                                              'ClmDiagnosisCode_5', \
                                              'ClmDiagnosisCode_6', \
                                              'ClmDiagnosisCode_7', \
                                              'ClmDiagnosisCode_8', \
                                              'ClmDiagnosisCode_9', \
                                              'ClmDiagnosisCode_10', \
                                              'ClmProcedureCode_1', \
                                              'ClmProcedureCode_2', \
                                              'ClmProcedureCode_3', \
                                              'ClmProcedureCode_4', \
                                              'ClmProcedureCode_5', \
                                              'ClmProcedureCode_6'], \
                                             keep = False)]

In [92]:
# totalpatient_d = Duplicated inpatient + Duplicated outpatient
totalpatient_d = pd.concat([inpatient_d, outpatient_d], axis=0)

In [93]:
duplicatedBeneID = totalpatient_d.groupby('Provider')['BeneID'].value_counts().reset_index(name = "ValueCount")
duplicatedBeneID = duplicatedBeneID.loc[duplicatedBeneID['ValueCount'] > 1]
duplicatedBeneID = duplicatedBeneID.groupby('Provider')['BeneID'].count().reset_index(name = "NumOfDuplicatedBeneID")

In [94]:
fraud7 = pd.merge(fraud7, duplicatedBeneID, on='Provider', how='left')

In [96]:
fraud7['NumOfDuplicatedBeneID'].fillna(0, inplace=True)

#### Adding count of duplicated attending physician per provider

In [97]:
duplicatedPhysician = totalpatient_d.groupby('Provider')['AttendingPhysician'].value_counts().reset_index(name = "ValueCount")
duplicatedPhysician = duplicatedPhysician.loc[duplicatedPhysician['ValueCount'] > 1]
duplicatedPhysician = duplicatedPhysician.groupby('Provider')['AttendingPhysician'].count().reset_index(name = "NumOfDuplicatedAttendingPhysician")

In [98]:
fraud7 = pd.merge(fraud7, duplicatedPhysician, on='Provider', how='left')
fraud7['NumOfDuplicatedAttendingPhysician'].fillna(0, inplace=True)

#### Adding count of duplicated claims per provider

In [100]:
duplicatedClaims = totalpatient_d.groupby('Provider').agg({'ClaimID' : 'count'}).reset_index()
duplicatedClaims = duplicatedClaims.rename({'ClaimID' : 'NumOfDuplicatedClaims'}, axis=1)

In [101]:
fraud7 = pd.merge(fraud7, duplicatedClaims, on='Provider', how='left')
fraud7['NumOfDuplicatedClaims'].fillna(0, inplace=True)

#### Adding the count of patients with different number of chronic conditions

In [107]:
fraud8 = fraud7.copy()

In [115]:
full_data2 = full_data1.copy()

In [116]:
full_data2 = full_data2.replace({'RenalDiseaseIndicator' : 'Y'}, 1)

full_data2['RenalDiseaseIndicator'] = full_data2['RenalDiseaseIndicator'].astype(int)

In [117]:
full_data2 = full_data2.replace({'ChronicCond_Alzheimer' : 2,'ChronicCond_Heartfailure' : 2,'ChronicCond_KidneyDisease' : 2,'ChronicCond_Cancer' : 2,'ChronicCond_ObstrPulmonary' : 2,'ChronicCond_Depression' : 2,'ChronicCond_Diabetes' : 2,'ChronicCond_IschemicHeart' : 2,'ChronicCond_Osteoporasis' : 2,'ChronicCond_rheumatoidarthritis' : 2,'ChronicCond_stroke' : 2}, 0)

In [118]:
full_data2['NumChronicCond'] = full_data2['ChronicCond_Alzheimer']+full_data2['ChronicCond_Heartfailure']+full_data2['ChronicCond_KidneyDisease']+full_data2['ChronicCond_Cancer']+full_data2['ChronicCond_ObstrPulmonary']+full_data2['ChronicCond_Depression']+full_data2['ChronicCond_Diabetes']+full_data2['ChronicCond_IschemicHeart']+full_data2['ChronicCond_Osteoporasis']+full_data2['ChronicCond_rheumatoidarthritis']+full_data2['ChronicCond_stroke']+full_data2['RenalDiseaseIndicator']

In [119]:
chronicConditions = full_data2.groupby(['Provider', 'BeneID'])['NumChronicCond'].value_counts().reset_index(name = 'NumChronicCondCount')
chronicConditions['NumChronicCondCount'] = chronicConditions['NumChronicCondCount'].apply(lambda x: 1 if x >= 1 else 0)
chronicConditions = chronicConditions.groupby('Provider')['NumChronicCond'].value_counts().reset_index(name = 'NumChronicCondCount')

In [120]:
chronicConditions = chronicConditions.groupby(['Provider', 'NumChronicCond']).NumChronicCondCount.sum()
chronicConditions = chronicConditions.unstack(level='NumChronicCond').reset_index()
chronicConditions = chronicConditions.rename({0 : 'ChronicCond0', 
                                              1 : 'ChronicCond1', 
                                              2 : 'ChronicCond2',
                                              3 : 'ChronicCond3',
                                              4 : 'ChronicCond4',
                                              5 : 'ChronicCond5',
                                              6 : 'ChronicCond6',
                                              7 : 'ChronicCond7',
                                              8 : 'ChronicCond8',
                                              9 : 'ChronicCond9',
                                              10 : 'ChronicCond10',
                                              11 : 'ChronicCond11',
                                              12 : 'ChronicCond12'}, axis=1)
chronicConditions.fillna(0, inplace=True)
chronicConditions

NumChronicCond,Provider,ChronicCond0,ChronicCond1,ChronicCond2,ChronicCond3,ChronicCond4,ChronicCond5,ChronicCond6,ChronicCond7,ChronicCond8,ChronicCond9,ChronicCond10,ChronicCond11,ChronicCond12
0,PRV51001,0.0,1.0,0.0,2.0,4.0,2.0,6.0,4.0,3.0,1.0,0.0,1.0,0.0
1,PRV51003,7.0,1.0,14.0,22.0,15.0,20.0,12.0,11.0,8.0,5.0,2.0,0.0,0.0
2,PRV51004,8.0,12.0,13.0,27.0,11.0,16.0,18.0,11.0,12.0,8.0,2.0,0.0,0.0
3,PRV51005,38.0,52.0,57.0,72.0,69.0,55.0,59.0,48.0,27.0,17.0,1.0,0.0,0.0
4,PRV51007,5.0,5.0,8.0,7.0,8.0,8.0,5.0,6.0,6.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,0.0,1.0,1.0,4.0,1.0,7.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0
5406,PRV57760,0.0,0.0,2.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5407,PRV57761,2.0,5.0,9.0,6.0,9.0,7.0,6.0,8.0,6.0,7.0,2.0,0.0,0.0
5408,PRV57762,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
fraud8 = pd.merge(fraud8, chronicConditions, on = 'Provider', how = 'left')

#### Adding counts of each gender of patients per provider

In [124]:
gender = full_data1.groupby(['Provider', 'BeneID', 'Gender'])['Gender'].count().reset_index(name = 'GenderCount')
gender['GenderCount'] = gender['GenderCount'].apply(lambda x: 1 if x >= 1 else 0)
gender = gender.groupby('Provider')['Gender'].value_counts().reset_index(name = 'GenderCount')

In [125]:
gender['Gender1'] = 0
gender['Gender2'] = 0

for index in gender.index:
    if gender['Gender'][index] == 1:
        gender['Gender1'][index] += gender['GenderCount'][index]
    if gender['Gender'][index] == 2:
        gender['Gender2'][index] += gender['GenderCount'][index]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [126]:
gender = gender.drop('Gender', axis = 1)
gender = gender.drop('GenderCount', axis = 1)

In [127]:
genderCount = gender.groupby('Provider').agg({'Gender1' : 'sum', 'Gender2' : 'sum'}).reset_index()

In [128]:
fraud8 = pd.merge(fraud8, genderCount, on = 'Provider', how = 'left')

#### Adding counts of each race of patients per provider

In [130]:
race = full_data1.groupby(['Provider', 'BeneID', 'Race'])['Race'].count().reset_index(name = 'RaceCount')
race['RaceCount'] = race['RaceCount'].apply(lambda x: 1 if x >= 1 else 0)
race = race.groupby('Provider')['Race'].value_counts().reset_index(name = 'RaceCount')

In [131]:
race['Race1'] = 0
race['Race2'] = 0
race['Race3'] = 0
race['Race5'] = 0

for index in race.index:
    if race['Race'][index] == 1:
        race['Race1'][index] += race['RaceCount'][index]
    if race['Race'][index] == 2:
        race['Race2'][index] += race['RaceCount'][index]
    if race['Race'][index] == 3:
        race['Race3'][index] += race['RaceCount'][index]
    if race['Race'][index] == 5:
        race['Race5'][index] += race['RaceCount'][index]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/i

In [132]:
race = race.drop('Race', axis = 1)
race = race.drop('RaceCount', axis = 1)

In [133]:
raceCount = race.groupby('Provider').agg({'Race1' : 'sum', 'Race2' : 'sum', 'Race3' : 'sum', 'Race5' : 'sum'}).reset_index()

In [134]:
fraud8 = pd.merge(fraud8, raceCount, on = 'Provider', how = 'left')

#### Replace na values with 0

In [136]:
fraud9 = fraud8.copy()

In [138]:
fraud9 = fraud9.replace(np.nan,0)

### Pickling Dataset

In [140]:
fraud9.to_pickle('train_dataset.pkl')